In [94]:
from zipfile37 import ZipFile
import yaml
import json
import flatdict as fd
import nion.swift.model.NDataHandler as nsnd
import h5py
import numpy as np
import uuid

In [2]:
def encode(uuid_: uuid.UUID, alphabet: str) -> str:
    result = str()
    uuid_int = uuid_.int
    while uuid_int:
        uuid_int, digit = divmod(uuid_int, len(alphabet))
        result += alphabet[digit]
    return result

# see https://github.com/nion-software/nionswift/blob/e95839c5602d009006ea88a648e5f78dc77c1ea4/
# nion/swift/model/Profile.py line 146 and following

def uuid_to_file_name(data_item_uuid_str: str) -> str:
    data_item_uuid_uuid = uuid.UUID(f'{data_item_uuid_str}')
    return f'data_{encode(data_item_uuid_uuid, "ABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890")}'
    # 25 character results

## Generate a dictionary with NeXus concept constraints from nionswift metadata dictionary

* Keyword is serialized path from metadata dictionary, if keyword is None this means we do not require the data element to have this metadata entry
* Value is a tuple with at least:
- a boolean (which is True if keyword is required to identify a match with NeXus concept)
- a NeXus variadic name path
- a dtype
- and only in the case that dtype is list, the number of required elements in the list (the dimensionality of the n-d data array to get dimension scale axes right)

In [115]:
metadata_constraints = {"type": str,
                        "uuid": str,
                        "created": str,
                        "data_shape": list,
                        "data_dtype": str,
                        "is_sequence": bool,
                        "dimensional_calibrations": list,
                        "data_modified": str,
                        "timezone": str,
                        "timezone_offset": str,
                        "metadata/hardware_source/hardware_source_id": str,
                        "version": int,
                        "modified": str }

nexus_concept_dict = {"ITULL": "NxImageSet(RealSpace)",
                      "IFLL": "NxImageSet(RealSpace)",
                      "IFL": None,
                      "ITUL": None,
                      "STUUE": "NxSpectrumSet(EELS)",
                      "STULLE": "NxSpectrumSet(EELS)",
                      "STULLUE": "NxSpectrumSet(OmegaQ)",
                      "SFLLUE": "NxSpectrumSet(OmegaQ)",
                      "SFLLE": "NxSpectrumSet(EELS)",
                      "SFUE": "NxSpectrumSet(EELS)",
                      "RFAA": "NxImage(AngularSpace)",
                      "RTUAA": "NxImage(AngularSpace)"}

def check_existence_of_required_fields(dct: dict, constraint_dct: dict) -> bool:
    """Checks if given dictionary has fields with values which match constraints."""
    flat_dct = fd.FlatDict(metadata_dict, delimiter='/')
    for keyword, dtyp in constraint_dct.items():
        if keyword not in flat_dct.keys():
            print(f"-->{keyword} not keyword")
            return False
        if not isinstance(flat_dct[keyword], dtyp):
            print(f"-->{keyword} not instance")
            return False
    return True

def identify_nexus_concept_key(dct: dict) -> str:
    """Identifies best candidate to map data/metadata on a NeXus concept."""
    nexus_concept_key = "UNKNOWN"
    if check_existence_of_required_fields(dct, metadata_constraints) is False:
        return nexus_concept_key
    lst_unit_catg = []
    for axis_dict in dct["dimensional_calibrations"]:  # inspect axes in sequence
        if isinstance(axis_dict, dict):
            if set(axis_dict.keys()) == set(["offset", "scale", "units"]):
                unit_arg = axis_dict["units"].lower()
                if unit_arg == "":
                    lst_unit_catg.append("U")
                elif unit_arg in ["nm"]:  # replace by pint to pick up on any length
                    lst_unit_catg.append("L")
                elif unit_arg in ["ev"]:  # replace by pint to pick up on any enery
                    lst_unit_catg.append("E")
                elif unit_arg in ["rad"]:  # replace by pint to pick up on angle unit
                    lst_unit_catg.append("A")
                else:
                    return nexus_concept_key
        set_unit_catg = set(lst_unit_catg)
            
        if "A" in set_unit_catg:
            nexus_concept_key = f"R{str(dct['is_sequence']).upper()[0:1]}{''.join(lst_unit_catg)}"
        elif "E" in set_unit_catg:
            nexus_concept_key = f"S{str(dct['is_sequence']).upper()[0:1]}{''.join(lst_unit_catg)}"
        elif "E" not in set_unit_catg:
            nexus_concept_key = f"I{str(dct['is_sequence']).upper()[0:1]}{''.join(lst_unit_catg)}"
        else:
            return nexus_concept_key
    return nexus_concept_key

In [116]:
def identify_dimension_scale_axes(dct: dict, nx_concept_key: str) -> list:
    """Create a list of dimension scale axes value, unit tuples."""
    axes = []
    if (check_existence_of_required_fields(dct, metadata_constraints) is False) \
            or nx_concept_key not in nexus_concept_dict.keys():
        return axes
    if nexus_concept_dict[nx_concept_key] is None:
        return axes
    if len(dct["dimensional_calibrations"]) != len(dct["data_shape"]):
        return axes
    
    for idx in np.arange(0, len(dct["dimensional_calibrations"])):
        nvalues = dct["data_shape"][idx]
        axis_dict = dct["dimensional_calibrations"][idx]
        if isinstance(nvalues, int) and isinstance(axis_dict, dict):
            if (nvalues > 0) and (set(axis_dict.keys()) == set(["offset", "scale", "units"])):
                # print(len(np.asarray(np.linspace(axis_dict["offset"], axis_dict["offset"]+nvalues*axis_dict["scale"], num=nvalues,endpoint=True), np.float64)))
                axes.append({"value": np.asarray(np.linspace(axis_dict["offset"], axis_dict["offset"]+nvalues*axis_dict["scale"], num=nvalues,endpoint=True), np.float64),
                       "unit": axis_dict["units"]})
    return axes

In [118]:
nexus_concept_glossary_voting = {}
nexus_class_glossary_voting = {}

from IPython.display import JSON
from jupyterlab_h5web import H5Web
# fnm = "data_24ZGOFFSF7NVOP322TBCKF0YF.h5"  # image stack ?
# fnm = "data_52BDPOOE2C7E4XADEYBLCTJXK.h5"  # prototype of eels stack with dectris ela detector
# fnm = "data_7EPPSHNUFKH6F6A4JCR45J03G.h5"  # also a 4d eels stack
fnm = "data_O4W29EVTX4DPTR33VRYHLI2QH.h5"  # a 5d eels stack ?
fnm = "data_SMO0N0BKKZZ8YRH85IL9QVFCH.h5"  # prototype of image stack 3d is_sequence true
# fnm = "data_YZ6EA0DDRKDJVZQWQP85LMT2I.h5"  # image stack ?
# fnm = "data_YZDB7C89BTP5AZQWEMKYZKT3E.h5"  # image stack ?
# fnm = "data_0ZG6LD26NSNHGRZT5IVW5NF9G.ndata"  # image stack with one 2d image is_sequence false
# fnm = "data_15Q9CYUOF1ZLKMGXJ8RIAY0YB.ndata"  # 
# fnm = "data_1X6R2F8BYAIW6FICNN1Z0BC8I.ndata"  # plain adf image 2d
# fnm = "data_85Q9O3MJY3CFUVJMGE7H62ETN.ndata"  # 4d eels stack
# fnm = "data_9BJWIRCXL3V52IO94A53X1T8O.ndata"  # prototype of an image stack for a ronchicam 2d
# fnm = "data_9BMSWFR4KA9K96G72AHVQGA4F.ndata"  # plain adf 2d
# fnm = "data_9K7VH6XP7RPNJUFI2HF03HSTH.ndata"  # plain adf 2d
# fnm = "data_KYJJLT8PUEBN6UXSPGESFND7E.ndata"  # plain adf 2d
# fnm = "data_NXBKMAPRJNSN79W3WHDMTWS6H.ndata"  # plain adf prototype image stack is different to 06PDLS3T because of subscan* relevant properties... 
# fnm = "data_O6PDLS3TMWVD4LURCYOBW2N9O.ndata"  # plain bright field image stack 2d
# fnm = "data_OUTU2CASRLFB6PMEIA2D5DI8F.ndata"  # eels summary
# fnm = "data_P7DVBM61LEQ1CMKPMZE2U90BH.ndata"  # eels stack 3d dectris ela but with additional metadata
# fnm = "data_WUOUUQPCGSB2FU0B0AONVAT2F.ndata"  # prototype of an image stack for a ronchicam 2d

fnms = ["data_24ZGOFFSF7NVOP322TBCKF0YF.h5",
        "data_52BDPOOE2C7E4XADEYBLCTJXK.h5",
        "data_7EPPSHNUFKH6F6A4JCR45J03G.h5",
        "data_O4W29EVTX4DPTR33VRYHLI2QH.h5",
        "data_SMO0N0BKKZZ8YRH85IL9QVFCH.h5",
        "data_YZ6EA0DDRKDJVZQWQP85LMT2I.h5",
        "data_YZDB7C89BTP5AZQWEMKYZKT3E.h5",
        "data_0ZG6LD26NSNHGRZT5IVW5NF9G.ndata",
        "data_15Q9CYUOF1ZLKMGXJ8RIAY0YB.ndata",
        "data_1X6R2F8BYAIW6FICNN1Z0BC8I.ndata",
        "data_85Q9O3MJY3CFUVJMGE7H62ETN.ndata",
        "data_9BJWIRCXL3V52IO94A53X1T8O.ndata",
        "data_9BMSWFR4KA9K96G72AHVQGA4F.ndata",
        "data_9K7VH6XP7RPNJUFI2HF03HSTH.ndata",
        "data_KYJJLT8PUEBN6UXSPGESFND7E.ndata",
        "data_NXBKMAPRJNSN79W3WHDMTWS6H.ndata",
        "data_O6PDLS3TMWVD4LURCYOBW2N9O.ndata",
        "data_OUTU2CASRLFB6PMEIA2D5DI8F.ndata",
        "data_P7DVBM61LEQ1CMKPMZE2U90BH.ndata",
        "data_WUOUUQPCGSB2FU0B0AONVAT2F.ndata"]

for fnm in fnms:
    if fnm.endswith(".h5"):
        h5r = h5py.File(fnm, "r")
        metadata = h5r["data"].attrs["properties"]
        # print(type(metadata))
        h5r.close()
        # print(type(metadata))
        metadata_dict = json.loads(metadata)
    elif fnm.endswith(".ndata"):
        with open(fnm, "r+b") as fp:
            local_files, dir_files, eocd = nsnd.parse_zip(fp)
            # print(local_files)
            # print(dir_files)
            # print(eocd)
        for offset, tpl in local_files.items():
            if tpl[0] == b'metadata.json':
                with open(fnm, "r+b") as fp:
                    fp.seek(offset)
                    # print(fp.tell())
                    metadata_dict = nsnd.read_json(fp, local_files, dir_files, b'metadata.json')
    else:
        print("Nothing to do :( !")
    # JSON(metadata_dict)

    print(fnm)
    nx_concept_key = identify_nexus_concept_key(metadata_dict)
    nx_concept_name = nexus_concept_dict[nx_concept_key]
    print(f"display_item is of concept type {nx_concept_key} which will be mapped onto {nx_concept_name}")
    # print(identify_dimension_scale_axes(metadata_dict, nx_concept_key))
    # yml = fd.FlatDict(metadata_dict, delimiter='/')
    
    # check for the entire set of display_items with each categorized as an instance of a specific concept KEY that these use all the same vocabulary 
    if nx_concept_key not in nexus_concept_glossary_voting.keys():
        nexus_concept_glossary_voting[nx_concept_key] = {}
        nexus_concept_glossary_voting[nx_concept_key]["TOTAL"] = 1
    else:
        nexus_concept_glossary_voting[nx_concept_key]["TOTAL"] += 1
    # check for the entire set of display_items with each categorized mapping to a specific concept NAME that these use all the same vocabulary, vote to identify all those metadata fields which are shared by all these instances of the same NAME to have a good example for a demo working with nionswift project files
    if nx_concept_name not in nexus_class_glossary_voting.keys():
        nexus_class_glossary_voting[nx_concept_name] = {}
        nexus_class_glossary_voting[nx_concept_name]["TOTAL"] = 1
    else:
        nexus_class_glossary_voting[nx_concept_name]["TOTAL"] += 1
    
    for serialized_path, value in yml.items():
    #     if serialized_path in ["dimensional_calibrations", "is_sequence"]:  # .startswith("intensity_calibration"):  # "metadata/hardware_source/hardware_source_id", , "is_sequence", "data_shape", ["category", "version"]
    #         print(f"{serialized_path}\t\t\t{yml[serialized_path]}")
        if serialized_path not in nexus_concept_glossary_voting[nx_concept_key].keys():
            nexus_concept_glossary_voting[nx_concept_key][serialized_path] = 1
        else:
            nexus_concept_glossary_voting[nx_concept_key][serialized_path] += 1
        
        if serialized_path not in nexus_class_glossary_voting[nx_concept_name].keys():
            nexus_class_glossary_voting[nx_concept_name][serialized_path] = 1
        else:
            nexus_class_glossary_voting[nx_concept_name][serialized_path] += 1
          
        # if isinstance(yml[serialized_path], list):
        #     print(len(yml[serialized_path]))

# see also https://github.com/nion-software/nionswift/blob/master/nion/swift/model/Model.py

data_24ZGOFFSF7NVOP322TBCKF0YF.h5
display_item is of concept type ITULL which will be mapped onto NxImageSet(RealSpace)
data_52BDPOOE2C7E4XADEYBLCTJXK.h5
display_item is of concept type STUUE which will be mapped onto NxSpectrumSet(EELS)
data_7EPPSHNUFKH6F6A4JCR45J03G.h5
display_item is of concept type STULLE which will be mapped onto NxSpectrumSet(EELS)
data_O4W29EVTX4DPTR33VRYHLI2QH.h5
display_item is of concept type STULLUE which will be mapped onto NxSpectrumSet(OmegaQ)
data_SMO0N0BKKZZ8YRH85IL9QVFCH.h5
display_item is of concept type ITULL which will be mapped onto NxImageSet(RealSpace)
data_YZ6EA0DDRKDJVZQWQP85LMT2I.h5
display_item is of concept type ITULL which will be mapped onto NxImageSet(RealSpace)
data_YZDB7C89BTP5AZQWEMKYZKT3E.h5
display_item is of concept type ITULL which will be mapped onto NxImageSet(RealSpace)
data_0ZG6LD26NSNHGRZT5IVW5NF9G.ndata
display_item is of concept type IFLL which will be mapped onto NxImageSet(RealSpace)
data_15Q9CYUOF1ZLKMGXJ8RIAY0YB.ndata
di

In [119]:
nexus_concept_glossary_voting
nexus_class_glossary_voting

{'NxImageSet(RealSpace)': {'TOTAL': 12,
  'type': 12,
  'uuid': 12,
  'created': 12,
  'data_shape': 12,
  'data_dtype': 12,
  'is_sequence': 12,
  'collection_dimension_count': 12,
  'datum_dimension_count': 12,
  'intensity_calibration/offset': 12,
  'intensity_calibration/scale': 12,
  'intensity_calibration/units': 12,
  'dimensional_calibrations': 12,
  'data_modified': 12,
  'timezone': 12,
  'timezone_offset': 12,
  'metadata/hardware_source/autostem/Acquisition:Binning': 12,
  'metadata/hardware_source/autostem/Acquisition:DarkMode': 12,
  'metadata/hardware_source/autostem/Acquisition:ExposureTime(s)': 12,
  'metadata/hardware_source/autostem/Acquisition:GainMode': 12,
  'metadata/hardware_source/autostem/Acquisition:IsFlipped': 12,
  'metadata/hardware_source/autostem/Acquisition:ReadoutBottom': 12,
  'metadata/hardware_source/autostem/Acquisition:ReadoutLeft': 12,
  'metadata/hardware_source/autostem/Acquisition:ReadoutRight': 12,
  'metadata/hardware_source/autostem/Acquisi

Types of datasets irrespective their individual metadata to infer a logic which can map opaque nion display_items on NeXus concepts.
The example from Benedikt includes data for the following imaging modalities:
For each case the following characteristics, is_sequence 1/0 and tuple of unit categories of the dimension scale axes for the data array

Images (hardware_source_id is superscan),
No dimension is energy
* ITULL, true, (unitless, length, length), sequence of images, it remains to distinguish is this now a bright field or a dark field, an HAADF image depends on the settings
* IFLL, false, (length, length), single image
* IFL ? line profile?
* ITUL ? sequence of line profiles?

Spectrum (hardware_source_id is dectris_ela)
One dimension is energy
* STUUE, true, (unitless, unitless, energy), sequence of summary spectra ?
* STULLE, true, (unitless, length, length, energy), sequence of EELS maps ?
* STULLUE, true, (unitless, length, length, unitless, energy), sequence of omega-q maps ?
* SFLLUE, false, (length, length, unitless, energy), single omega-q map?
* SFLLE, false, (length, length, energy)
* SFUE ?

Ronchicam (hardware_source_id is orca1):
At least one dimension is angle
* RFAA, false, (angle, angle)
* RTUAA ?
* All types of low

Likely other combinations are possible such as specific lower-dimensional sections, but examples how this would look in metadata.json are not included in Benedikts dataset.<br>
In the above-mentioned lists these are marked with question mark.<br>
The field triplet intensity_calibration/(offset, scale, units) were always (0.0, 1.0) and either "counts" or ""<br>
The field category has either persistent or temporary what is the difference, can it be related to post-processed or collected data?<br>
The field version was always 13 in Benedikt's example.<br>

However images differ as some are in real-space and some in angular space.<br>
But it is even more tricky what makes an image an annular dark field image?<br>
Being an image alone is not sufficient also specific imaging mode and settings have to be met, i.e. a specific set of vendor-jargon field values and datatypes.<br>
Furthermore, recall that nionswift is a generic image processing software.<br>
One can load a picture from you travelling with your colleagues on a business trip.<br>
This qualifies of type IFLL but it is for sure not an annular dark field image.<br>

The dictionary of supported NeXus concept keys

In [85]:
eels_stack = {"type": (True, None, str),
              "uuid": (True, None, str),
              "created": (True, None, str),
              "data_shape": (True, None, list, 3),
              "data_dtype": (True, None, float),
              "is_sequence": (True, None, bool),
              "collection_dimension_count": (True, None, int),
              "datum_dimension_count": (True, None, int),
              "intensity_calibration/offset": (True, None, float),
              "intensity_calibration/scale": (True, None, None),
              "intensity_calibration/units": (True, None, str),
              "dimensional_calibrations": (True, None, list, 3),
              "data_modified": (True, None, str),
              "timezone": (True, None, None),
              "timezone_offset": (True, None, str),
              "metadata/hardware_source/counts_per_electron": (True, None, float),
              "metadata/hardware_source/hardware_source_id": (True, None, str),
              "metadata/hardware_source/hardware_source_name": (True, None, None),
              "metadata/hardware_source/exposure": (True, None, float),
              "metadata/hardware_source/binning": (True, None, int),
              "metadata/hardware_source/signal_type": (True, None, str),
              "metadata/instrument/high_tension": (True, None, float),
              "metadata/instrument/defocus": (True, None, float),
              "metadata/instrument/ImageRonchigram/EHT": (True, None, float),
              "metadata/instrument/ImageRonchigram/MajorOL": (True, None, float),
              "metadata/instrument/ImageRonchigram/StageOutA": (True, None, float),
              "metadata/instrument/ImageRonchigram/StageOutB": (True, None, float),
              "metadata/instrument/ImageRonchigram/StageOutX": (True, None, float),
              "metadata/instrument/ImageRonchigram/StageOutY": (True, None, float),
              "metadata/instrument/ImageRonchigram/StageOutZ": (True, None, float),
              "metadata/instrument/ImageRonchigram/probe_ha": (True, None, float),
              "metadata/instrument/ImageRonchigram/SuperFEG.^EmissionCurrent": (True, None, float),
              "metadata/instrument/ImageRonchigram/LastTuneCurrent": (True, None, float),
              "metadata/instrument/ImageRonchigram/C10": (True, None, float),
              "metadata/instrument/ImageRonchigram/C12.a": (True, None, float),
              "metadata/instrument/ImageRonchigram/C12.b": (True, None, float),
              "metadata/instrument/ImageRonchigram/C21.a": (True, None, float),
              "metadata/instrument/ImageRonchigram/C21.b": (True, None, float),
              "metadata/instrument/ImageRonchigram/C23.a": (True, None, float),
              "metadata/instrument/ImageRonchigram/C23.b": (True, None, float),
              "metadata/instrument/ImageRonchigram/C30": (True, None, float),
              "metadata/instrument/ImageRonchigram/C32.a": (True, None, float),
              "metadata/instrument/ImageRonchigram/C32.b": (True, None, float),
              "metadata/instrument/ImageRonchigram/C34.a": (True, None, float),
              "metadata/instrument/ImageRonchigram/C34.b": (True, None, float),
              "metadata/instrument/ImageRonchigram/C50": (True, None, float),
              "metadata/MultiAcquire.parameters/index": (True, None, int),
              "metadata/MultiAcquire.parameters/offset_x": (True, None, float),
              "metadata/MultiAcquire.parameters/offset_y": (True, None, float),
              "metadata/MultiAcquire.parameters/exposure_ms": (True, None, float),
              "metadata/MultiAcquire.parameters/frames": (True, None, int),
              "metadata/MultiAcquire.parameters/start_ev": (True, None, float),
              "metadata/MultiAcquire.parameters/end_ev": (True, None, float),
              "metadata/MultiAcquire.settings/x_shifter": (True, None, str),
              "metadata/MultiAcquire.settings/blanker": (True, None, str),
              "metadata/MultiAcquire.settings/x_shift_delay": (True, None, float),
              "metadata/MultiAcquire.settings/focus": (True, None, None),
              "metadata/MultiAcquire.settings/focus_delay": (True, None, int),
              "metadata/MultiAcquire.settings/auto_dark_subtract": (True, None, bool),
              "metadata/MultiAcquire.settings/blanker_delay": (True, None, float),
              "metadata/MultiAcquire.settings/sum_frames": (True, None, bool),
              "metadata/MultiAcquire.settings/camera_hardware_source_id": (True, None, str),
              "metadata/MultiAcquire.settings/use_multi_eels_calibration": (True, None, bool),
              "metadata/MultiAcquire.settings/shift_each_sequence_slice": (True, None, bool),
              "metadata/MultiAcquire.settings/y_shifter": (True, None, None),
              "metadata/MultiAcquire.settings/x_units_per_ev": (True, None, float),
              "metadata/MultiAcquire.settings/y_units_per_px": (True, None, float),
              "metadata/MultiAcquire.settings/y_shift_delay": (True, None, float),
              "metadata/MultiAcquire.settings/saturation_value": (True, None, int),
              "metadata/MultiAcquire.settings/y_align": (True, None, bool),
              "metadata/MultiAcquire.settings/stitch_spectra": (True, None, bool),
              "title": (True, None, str),
              "session_id": (True, None, str),
              "session": (True, None, dict),
              "category": (True, None, str),
              "version": (True, None, int),
              "modified": (True, None, str) }

In [ ]:
image_stack = {"type": (True, None, str),
               "uuid": (True, None, str),
               "created": (True, None, str),
               "data_shape": (True, None, list, 3),
               "data_dtype": (True, None, float),
               "is_sequence": (True, None, bool),
               "collection_dimension_count": (True, None, int),
               "datum_dimension_count": (True, None, int),
               "intensity_calibration/offset": (True, None, float),
               "intensity_calibration/scale": (True, None, float),
               "intensity_calibration/units": (True, None, None),
               "dimensional_calibrations": (True, None, list, 3),
               "data_modified": (True, None, str),
               "timezone": (True, None, str),
               "timezone_offset": (True, None, str),
               "metadata/instrument/high_tension": (True, None, float),
               "metadata/instrument/defocus": (True, None, float),
               "metadata/instrument/ImageScanned/EHT": (True, None, float),
               "metadata/instrument/ImageScanned/PMTBF_gain": (True, None, float),
               "metadata/instrument/ImageScanned/PMTDF_gain": (True, None, float),
               "metadata/instrument/ImageScanned/StageOutA": (True, None, float),
               "metadata/instrument/ImageScanned/StageOutB": (True, None, float),
               "metadata/instrument/ImageScanned/StageOutX": (True, None, float),
               "metadata/instrument/ImageScanned/StageOutY": (True, None, float),
               "metadata/instrument/ImageScanned/StageOutZ": (True, None, float),
               "metadata/instrument/ImageScanned/C10": (True, None, float),
               "metadata/instrument/ImageScanned/C12.a": (True, None, float),
               "metadata/instrument/ImageScanned/C12.b": (True, None, float),
               "metadata/instrument/ImageScanned/C21.a": (True, None, float),
               "metadata/instrument/ImageScanned/C21.b": (True, None, float),
               "metadata/instrument/ImageScanned/C23.a": (True, None, float),
               "metadata/instrument/ImageScanned/C23.b": (True, None, float),
               "metadata/instrument/ImageScanned/C30": (True, None, float),
               "metadata/instrument/ImageScanned/C32.a": (True, None, float),
               "metadata/instrument/ImageScanned/C32.b": (True, None, float),
               "metadata/instrument/ImageScanned/C34.a": (True, None, float),
               "metadata/instrument/ImageScanned/C34.b": (True, None, float),
               "metadata/instrument/ImageScanned/C50": (True, None, float),
               "metadata/instrument/ImageScanned/C1 ConstW": (True, None, float),
               "metadata/instrument/ImageScanned/C2 ConstW": (True, None, float),
               "metadata/instrument/ImageScanned/C3 ConstW": (True, None, float),
               "metadata/instrument/ImageScanned/PMT2_gain": (True, None, float),
               "metadata/instrument/ImageScanned/SuperFEG.^EmissionCurrent": (True, None, float),
               "metadata/instrument/ImageScanned/G_2Db": (True, None, float),
               "metadata/instrument/ImageScanned/LastTuneCurrent": (True, None, float),
               "metadata/instrument/ImageScanned/probe_ha": (True, None, float),
               "metadata/instrument/ImageScanned/HAADF_Inner_ha": (True, None, float),
               "metadata/instrument/ImageScanned/HAADF_Outer_ha": (True, None, float),
               "metadata/instrument/ImageScanned/GeometricProbeSize": (True, None, float),
               "metadata/scan/hardware_source_id": (True, None, str),
               "metadata/scan/hardware_source_name": (True, None, str),
               "metadata/scan/scan_id": (True, None, str),
               "metadata/scan/center_x_nm": (True, None, float),
               "metadata/scan/center_y_nm": (True, None, float),
               "metadata/scan/fov_nm": (True, None, float),
               "metadata/scan/rotation": (True, None, float),
               "metadata/scan/rotation_deg": (True, None, float),
               "metadata/scan/scan_context_size": (True, None, list, 2),
               "metadata/scan/scan_size": (True, None, list, 2),
               "metadata/scan/scan_device_parameters/size": (True, None, list, 2),
               "metadata/scan/scan_device_parameters/center_nm": (True, None, list, 2),
               "metadata/scan/scan_device_parameters/pixel_time_us": (True, None, float),
               "metadata/scan/scan_device_parameters/fov_nm": (True, None, float),
               "metadata/scan/scan_device_parameters/rotation_rad": (True, None, float),
               "metadata/scan/scan_device_parameters/external_clock_wait_time_ms": (True, None, float),
               "metadata/scan/scan_device_parameters/external_clock_mode": (True, None, int),
               "metadata/scan/scan_device_parameters/external_scan_mode": (True, None, int),
               "metadata/scan/scan_device_parameters/external_scan_ratio": (True, None, float),
               "metadata/scan/scan_device_parameters/ac_line_sync": (True, None, bool),
               "metadata/scan/scan_device_parameters/ac_frame_sync": (True, None, bool),
               "metadata/scan/scan_device_parameters/flyback_time_us": (True, None, float),
               "metadata/scan/scan_device_properties/ac_line_sync": (True, None, float),
               "metadata/scan/scan_device_properties/calibration_style": (True, None, str),
               "metadata/scan/scan_device_properties/center_x_nm": (True, None, float),
               "metadata/scan/scan_device_properties/center_y_nm": (True, None, float),
               "metadata/scan/scan_device_properties/flyback_time_us": (True, None, float),
               "metadata/scan/scan_device_properties/fov_nm": (True, None, float),
               "metadata/scan/scan_device_properties/line_time_us": (True, None, float),
               "metadata/scan/scan_device_properties/pixel_time_us": (True, None, float),
               "metadata/scan/scan_device_properties/pixels_x": (True, None, float),
               "metadata/scan/scan_device_properties/pixels_y": (True, None, float),
               "metadata/scan/scan_device_properties/requested_pixel_time_us": (True, None, float),
               "metadata/scan/scan_device_properties/rotation_deg": (True, None, float),
               "metadata/scan/scan_device_properties/rotation_rad": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 0 DAC 0": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 0 DAC 1": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 0 DAC 10": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 0 DAC 11": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 0 DAC 2": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 0 DAC 3": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 0 DAC 4": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 0 DAC 5": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 0 DAC 6": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 0 DAC 7": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 0 DAC 8": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 0 DAC 9": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 0 Relay": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 1 DAC 0": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 1 DAC 1": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 1 DAC 10": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 1 DAC 11": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 1 DAC 2": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 1 DAC 3": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 1 DAC 4": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 1 DAC 5": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 1 DAC 6": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 1 DAC 7": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 1 DAC 8": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 1 DAC 9": (True, None, float),
               "metadata/scan/scan_device_properties/mag_boards/MagBoard 1 Relay": (True, None, float),
               "metadata/hardware_source/hardware_source_id": (True, None, str),
               "metadata/hardware_source/hardware_source_name": (True, None, str),
               "metadata/hardware_source/exposure": (True, None, float),
               "metadata/hardware_source/channel_id": (True, None, str),
               "metadata/hardware_source/channel_name": (True, None, str),
               "metadata/hardware_source/pixel_time_us": (True, None, float),
               "metadata/hardware_source/line_time_us": (True, None, float),
               "title": (True, None, str),
               "session_id": (True, None, str),
               "session": (True, None, dict),
               "category": (True, None, str),
               "version": (True, None, int),
               "modified": (True, None, str) }

This defines the constraints based on which we decide if a given display_item with its metadata and data indeed describes an instance of an NXevent_data_em/eels stack.

# TODO:
* Add variadic path names for NeXus
* Add logic which tests if a given instance of a display_item matches a concept
* Add writer mapping the specific concept

In [75]:
import numpy as np
offset = -69.09271876220336
scale = 0.6974271876220337
n = 1028
np.asarray(np.linspace(offset, offset + n*scale,num=n, endpoint=True), np.float64)

array([-69.09271876, -68.39461248, -67.6965062 , ..., 646.46621755,
       647.16432383, 647.86243011])

## The actual compressed nionswift project parser skeleton

In [120]:
file_name = "2022-02-18_Metadata_Kuehbach.nszip"
proj_file_names = []  # to assure that there is exactly one *.nsproj file only to parse from
ndata_file_dict = {}  # just get the *.ndata files irrespective whether they will be parsed later
hdf_file_dict = {}  # just get the *.h5 files irrespective whether they will be interpreted

with ZipFile(file_name) as zip_file_hdl:
    # print(zip_file_hdl.namelist())
    # identify first the content in this archive
    # ASSUME that pattern have numeral components in their file name
    # zip_content_table = {}
    for file in zip_file_hdl.namelist():
        if file.endswith(".h5"):
            key = file[file.rfind("/")+1:].replace(".h5", "")
            if key not in hdf_file_dict:
                hdf_file_dict[key] = file
        elif file.endswith(".ndata"):
            key = file[file.rfind("/")+1:].replace(".ndata", "")
            if key not in ndata_file_dict:
                ndata_file_dict[key] = file
        elif file.endswith(".nsproj"):
            proj_file_names.append(file)
        else:
            continue
print(f"Are there duplicated keys for ndata and hdf files? {ndata_file_dict.keys() == hdf_file_dict.keys()}")
print(proj_file_names)
print(ndata_file_dict)
print(hdf_file_dict)
    #    print(file)
    #     keyword = str(np.uint64(re.sub('[^0-9]', '', file)))
    #     if len(keyword) > 0 and keyword not in zip_content_table:
    #         zip_content_table[keyword] = file
    #     else:
    #         print("WARNING::Automatically derived name is not unique!")
    #         return template

Are there duplicated keys for ndata and hdf files? False
['2022-02-18_Metadata_Kuehbach.nsproj']
{'data_0ZG6LD26NSNHGRZT5IVW5NF9G': '2022-02-18_Metadata_Kuehbach Data/2022/02/18/20220218-140947/data_0ZG6LD26NSNHGRZT5IVW5NF9G.ndata', 'data_15Q9CYUOF1ZLKMGXJ8RIAY0YB': '2022-02-18_Metadata_Kuehbach Data/2022/02/18/20220218-140947/data_15Q9CYUOF1ZLKMGXJ8RIAY0YB.ndata', 'data_1X6R2F8BYAIW6FICNN1Z0BC8I': '2022-02-18_Metadata_Kuehbach Data/2022/02/18/20220218-140947/data_1X6R2F8BYAIW6FICNN1Z0BC8I.ndata', 'data_85Q9O3MJY3CFUVJMGE7H62ETN': '2022-02-18_Metadata_Kuehbach Data/2022/02/18/20220218-140947/data_85Q9O3MJY3CFUVJMGE7H62ETN.ndata', 'data_9BJWIRCXL3V52IO94A53X1T8O': '2022-02-18_Metadata_Kuehbach Data/2022/02/18/20220218-140947/data_9BJWIRCXL3V52IO94A53X1T8O.ndata', 'data_9BMSWFR4KA9K96G72AHVQGA4F': '2022-02-18_Metadata_Kuehbach Data/2022/02/18/20220218-140947/data_9BMSWFR4KA9K96G72AHVQGA4F.ndata', 'data_9K7VH6XP7RPNJUFI2HF03HSTH': '2022-02-18_Metadata_Kuehbach Data/2022/02/18/20220218-140

In [121]:
with ZipFile(file_name) as zip_file_hdl:
    with zip_file_hdl.open(proj_file_names[0]) as file_hdl:
        # with open(file_name, 'r') as stream:
        yml = fd.FlatDict(yaml.safe_load(file_hdl), delimiter='/')
        # for serialized_path in yml.keys():
        #     print(f"{serialized_path}\t\t\t{yml[serialized_path]}")
        for entry in yml["display_items"]:
            print(entry)

{'type': 'display_item', 'uuid': '0256be77-7f65-401b-a1ba-d31064e37dfb', 'created': '2022-02-18T13:09:47.976613', 'calibration_style_id': 'calibrated', 'display_properties': {}, 'graphics': [], 'display_layers': [{'data_row': 0, 'fill_color': '#1E90FF', 'data_index': 0}], 'display_data_channels': [{'type': 'display_data_channel', 'uuid': '9c48aa2a-c910-4200-a625-0f01a09a3578', 'brightness': 0.0, 'contrast': 1.0, 'adjustments': [], 'sequence_index': 0, 'collection_index': [0, 0, 0], 'slice_center': 0, 'slice_width': 1, 'data_item_reference': 'fc37bb8f-9de2-4cbd-9eaa-cf3fdc22a356', 'modified': '2022-02-18T20:27:44.597242'}], 'modified': '2022-08-31T21:51:34.436866'}
{'type': 'display_item', 'uuid': '286f567d-3dac-4d7c-83ef-31108d68dc3f', 'created': '2022-02-18T13:10:07.481671', 'calibration_style_id': 'calibrated', 'display_properties': {'image_position': [0.4537037037037039, 0.736111111111111], 'image_canvas_mode': 'custom'}, 'graphics': [{'type': 'point-graphic', 'uuid': '4fb6d52e-b6ba

In [122]:
for itm in yml["display_items"]:
    if set(["type", "uuid", "created", "display_data_channels"]).issubset(itm.keys()):
        if len(itm["display_data_channels"]) == 1:
            if "data_item_reference" in itm["display_data_channels"][0].keys():
                key = uuid_to_file_name(itm["display_data_channels"][0]["data_item_reference"])  # file_name without the mime type
                if key in ndata_file_dict:
                    print(f"Key {key} is *.ndata maps to {ndata_file_dict[key]}")
                    with ZipFile(file_name) as zip_file_hdl:
                        # entering the ndata parser ...
                        with zip_file_hdl.open(ndata_file_dict[key]) as fp:
                            local_files, dir_files, eocd = nsnd.parse_zip(fp)
                            print(local_files)
                            print(dir_files)
                            print(eocd)
                            # check that there is metadata.json in local_files
                            # check if these metadata are matching exactly to one known NeXus concept of NXem
                            # if so, check that there is a data.npy and pass both to the concept writer
                elif key in hdf_file_dict:
                    print(f"Key {key} is *.h5 maps to {hdf_file_dict[key]}")
                    with ZipFile(file_name) as zip_file_hdl:
                        with zip_file_hdl.open(hdf_file_dict[key]) as fp:
                            h5r = h5py.File(fp, "r")
                            metadata = h5r["data"].attrs["properties"]
                            # print(type(metadata))
                            h5r.close()
                            # print(type(metadata))
                            metadata_dict = json.loads(metadata)
                            for keyword, value in metadata_dict.items():
                                print(f"{keyword}\t\t\t{value}")
                            # print(metadata_dict)
                            # with open(f"{key}.h5.metadata.json", "w") as f:
                            #     json.dump(metadata_dict, f, ensure_ascii=False)
                            # check that this metadata_dict was retrieveable
                            # check if this metadata_dict matches exactly to one known NeXus concept of NXem
                            # if so, check that there are data matching this concept and pass to the concept writer
                else:
                    print(f"Key {key} has no corresponding data file")

Key data_9BJWIRCXL3V52IO94A53X1T8O is *.ndata maps to 2022-02-18_Metadata_Kuehbach Data/2022/02/18/20220218-140947/data_9BJWIRCXL3V52IO94A53X1T8O.ndata
{0: (b'data.npy', 38, 16777344, 4199220949), 16777382: (b'metadata.json', 16777425, 1829, 1707453876)}
{b'data.npy': (16779254, 0), b'metadata.json': (16779308, 16777382)}
(16779367, 16779254)
Key data_NXBKMAPRJNSN79W3WHDMTWS6H is *.ndata maps to 2022-02-18_Metadata_Kuehbach Data/2022/02/18/20220218-140947/data_NXBKMAPRJNSN79W3WHDMTWS6H.ndata
{0: (b'data.npy', 38, 368, 3907305299), 406: (b'metadata.json', 449, 4262, 2414648985)}
{b'data.npy': (4711, 0), b'metadata.json': (4765, 406)}
(4824, 4711)
Key data_WUOUUQPCGSB2FU0B0AONVAT2F is *.ndata maps to 2022-02-18_Metadata_Kuehbach Data/2022/02/18/20220218-140947/data_WUOUUQPCGSB2FU0B0AONVAT2F.ndata
{0: (b'data.npy', 38, 16777344, 3784912480), 16777382: (b'metadata.json', 16777425, 1828, 3024408658)}
{b'data.npy': (16779253, 0), b'metadata.json': (16779307, 16777382)}
(16779366, 16779253)
K

***
## Debugging

In [36]:
# yml["display_items"][0]["display_data_channels"]
# yml["display_items"][0]["display_data_channels"][0]["data_item_reference"]

[{'type': 'display_data_channel',
  'uuid': '9c48aa2a-c910-4200-a625-0f01a09a3578',
  'brightness': 0.0,
  'contrast': 1.0,
  'adjustments': [],
  'sequence_index': 0,
  'collection_index': [0, 0, 0],
  'slice_center': 0,
  'slice_width': 1,
  'data_item_reference': 'fc37bb8f-9de2-4cbd-9eaa-cf3fdc22a356',
  'modified': '2022-02-18T20:27:44.597242'}]

In [20]:
fnm = "2022-02-18_Metadata_Kuehbach Data/2022/02/18/20220218-140947/data_0ZG6LD26NSNHGRZT5IVW5NF9G.ndata"
print(fnm[fnm.rfind("/")+1:].replace(".ndata", ""))

data_0ZG6LD26NSNHGRZT5IVW5NF9G
